In [12]:
import pandas as pd

In [14]:
experiment = pd.read_csv('data/test1_trial_2017_May_22_1013.csv')

In [26]:
experiment.columns

Index(['noise', 'location', 'correspondingAudio', 'combinedTrials.thisRepN',
       'combinedTrials.thisTrialN', 'combinedTrials.thisN',
       'combinedTrials.thisIndex', 'WelcomeKey.keys', 'WelcomeKey.rt',
       'key_resp_2.keys', 'key_resp_2.rt', 'CombinedAccept.keys',
       'CombinedAccept.rt', 'CombComparisonResponse.keys',
       'CombComparisonResponse.corr', 'CombComparisonResponse.rt',
       'participant', 'age', 'frameRate', 'sex', 'expName', 'date',
       'Unnamed: 22'],
      dtype='object')

In [42]:
def preprocess_data(filename):
    experiment = pd.read_csv('data' + filename)
    

SyntaxError: unexpected EOF while parsing (<ipython-input-42-4e227e282d8d>, line 1)

In [41]:
combined = experiment[pd.notnull(experiment['CombComparisonResponse.keys'])]

,noise,location,correspondingAudio,combinedTrials.thisRepN,combinedTrials.thisTrialN,combinedTrials.thisN,combinedTrials.thisIndex,WelcomeKey.keys,WelcomeKey.rt,key_resp_2.keys,...,CombComparisonResponse.keys,CombComparisonResponse.corr,CombComparisonResponse.rt,participant,age,frameRate,sex,expName,date,Unnamed: 22
3,0.62,"[2.25, 0]",/sounds/legit/right30.wav,0.0,0.0,0.0,9.0,NaN,NaN,NaN,...,right,0.0,5.087143,test1,24,59.941235,f,trial,2017_May_22_1013,NaN
4,0.10,"[-6.75, 0]",/sounds/legit/left100.wav,0.0,1.0,1.0,30.0,NaN,NaN,NaN,...,right,0.0,4.971276,test1,24,59.941235,f,trial,2017_May_22_1013,NaN
5,0.10,"[-6.75, 0]",/sounds/legit/left100.wav,0.0,2.0,2.0,30.0,NaN,NaN,NaN,...,left,0.0,1.020282,test1,24,59.941235,f,trial,2017_May_22_1013,NaN
6,0.23,"[4.5, 0]",/sounds/legit/right60.wav,0.0,3.0,3.0,11.0,NaN,NaN,NaN,...,right,0.0,0.701864,test1,24,59.941235,f,trial,2017_May_22_1013,NaN
7,0.62,"[6.75, 0]",/sounds/legit/right100.wav,0.0,4.0,4.0,19.0,NaN,NaN,NaN,...,right,0.0,0.536533,test1,24,59.941235,f,trial,2017_May_22_1013,NaN
8,0.23,"[2.25, 0]",/sounds/legit/right30.wav,0.0,5.0,5.0,6.0,NaN,NaN,NaN,...,right,0.0,0.518686,test1,24,59.941235,f,trial,2017_May_22_1013,NaN
9,0.10,"[4.5, 0]",/sounds/legit/right60.wav,0.0,6.0,6.0,10.0,NaN,NaN,NaN,...,right,0.0,0.602686,test1,24,59.941235,f,trial,2017_May_22_1013,NaN
10,0.36,"[6.75, 0]",/sounds/legit/right100.wav,0.0,7.0,7.0,17.0,NaN,NaN,NaN,...,right,0.0,0.551786,test1,24,59.941235,f,trial,2017_May_22_1013,NaN
11,0.36,"[4.5, 0]",/sounds/legit/right60.wav,0.0,8.0,8.0,12.0,NaN,NaN,NaN,...,right,0.0,0.585573,test1,24,59.941235,f,trial,2017_May_22_1013,NaN
12,0.36,"[6.75, 0]",/sounds/legit/right100.wav,0.0,9.0,9.0,17.0,NaN,NaN,NaN,...,right,0.0,0.535843,test1,24,59.941235,f,trial,2017_May_22_1013,NaN


In [34]:
experiment['location']

0             NaN
1             NaN
2             NaN
3       [2.25, 0]
4      [-6.75, 0]
5      [-6.75, 0]
6        [4.5, 0]
7       [6.75, 0]
8       [2.25, 0]
9        [4.5, 0]
10      [6.75, 0]
11       [4.5, 0]
12      [6.75, 0]
13     [-6.75, 0]
14       [4.5, 0]
15     [-6.75, 0]
16      [2.25, 0]
17       [4.5, 0]
18         [0, 0]
19      [-4.5, 0]
20         [0, 0]
21         [0, 0]
22      [-4.5, 0]
23      [-4.5, 0]
24      [2.25, 0]
25      [-4.5, 0]
26       [4.5, 0]
27      [2.25, 0]
28      [2.25, 0]
29         [0, 0]
          ...    
115     [6.75, 0]
116     [-4.5, 0]
117     [6.75, 0]
118        [0, 0]
119     [6.75, 0]
120    [-6.75, 0]
121    [-2.25, 0]
122     [2.25, 0]
123     [-4.5, 0]
124     [-4.5, 0]
125    [-6.75, 0]
126        [0, 0]
127    [-2.25, 0]
128    [-2.25, 0]
129     [-4.5, 0]
130      [4.5, 0]
131      [4.5, 0]
132     [6.75, 0]
133    [-6.75, 0]
134    [-2.25, 0]
135     [2.25, 0]
136      [4.5, 0]
137    [-6.75, 0]
138      [4.5, 0]
139     [6

In [35]:
experiment['noise']

0       NaN
1       NaN
2       NaN
3      0.62
4      0.10
5      0.10
6      0.23
7      0.62
8      0.23
9      0.10
10     0.36
11     0.36
12     0.36
13     0.10
14     0.23
15     0.10
16     0.10
17     0.36
18     0.10
19     0.62
20     0.49
21     0.23
22     0.62
23     0.23
24     0.62
25     0.62
26     0.62
27     0.62
28     0.10
29     0.62
       ... 
115    0.36
116    0.23
117    0.49
118    0.10
119    0.62
120    0.49
121    0.10
122    0.10
123    0.36
124    0.10
125    0.62
126    0.49
127    0.62
128    0.49
129    0.23
130    0.36
131    0.49
132    0.23
133    0.36
134    0.10
135    0.49
136    0.23
137    0.62
138    0.36
139    0.62
140    0.10
141    0.49
142    0.49
143    0.10
144    0.49
Name: noise, dtype: float64